The goal of this notebook is to take the ad hoc implementation in `MIPExample1.ipynb` and vectorize the operations such that any size Tyche problem can be converted to a MIP and solved.

In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath("../src"))

In [3]:
import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb
import tyche             as ty

from copy            import deepcopy
from IPython.display import Image 

In [4]:
import cProfile
import timeit

In [5]:
from mip import Model, minimize, BINARY, xsum

In [6]:
designs = ty.Designs("data")
investments = ty.Investments("data")
designs.compile()
tranche_results = investments.evaluate_tranches(designs, sample_count=250)
results = investments.tranches.join(tranche_results.summary)
evaluator = ty.Evaluator(investments.tranches, tranche_results.summary)

Get the wide-format interpolated elicitation data from the Tyche Evaluator and reset the multi-level index.

In [112]:
wide_multi = evaluator.evaluate_corners_wide()
wide = evaluator.evaluate_corners_wide().reset_index()

In [113]:
wide_multi

Index                                                                                         Capital  \
CIGS      CdTe      GaAs      InGaP     Perovskite Polysilicon Power Electronics Soft Costs             
0.0       0.0       0.0       0.0       0.0        0.0         0.0               0.0        -1.575208   
                                                                                 1000000.0  -1.486433   
                                                                                 5000000.0  -1.319666   
                                                               1000000.0         0.0        -1.505137   
                                                                                 1000000.0  -1.416362   
...                                                                                               ...   
3000000.0 6000000.0 7500000.0 9500000.0 9500000.0  4500000.0   1000000.0         1000000.0   3.288859   
                                                                                 5000000.0   3.455626   
                                                               5000000.0         0.0         3.290173   
                                                                                 1000000.0   3.378947   
                                                                                 5000000.0   3.545714   

Index                                                                                        Efficiency  \
CIGS      CdTe      GaAs      InGaP     Perovskite Polysilicon Power Electronics Soft Costs               
0.0       0.0       0.0       0.0       0.0        0.0         0.0               0.0           2.065006   
                                                                                 1000000.0     2.065172   
                                                                                 5000000.0     2.065531   
                                                               1000000.0         0.0           2.064491   
                                                                                 1000000.0     2.064657   
...                                                                                                 ...   
3000000.0 6000000.0 7500000.0 9500000.0 9500000.0  4500000.0   1000000.0         1000000.0     2.336959   
                                                                                 5000000.0     2.337318   
                                                               5000000.0         0.0           2.338137   
                                                                                 1000000.0     2.338304   
                                                                                 5000000.0     2.338663   

Index                                                                                             GHG  \
CIGS      CdTe      GaAs      InGaP     Perovskite Polysilicon Power Electronics Soft Costs             
0.0       0.0       0.0       0.0       0.0        0.0         0.0               0.0        -0.003592   
                                                                                 1000000.0  -0.003592   
                                                                                 5000000.0  -0.003592   
                                                               1000000.0         0.0        -0.003592   
                                                                                 1000000.0  -0.003592   
...                                                                                               ...   
3000000.0 6000000.0 7500000.0 9500000.0 9500000.0  4500000.0   1000000.0         1000000.0  -0.003592   
                                                                                 5000000.0  -0.003592   
                                                               5000000.0         0.0        -0.003592   
                                                                                 100000

In [126]:
wide_multi.index

MultiIndex([(      0.0,       0.0,       0.0,       0.0,       0.0, ...),
            (      0.0,       0.0,       0.0,       0.0,       0.0, ...),
            (      0.0,       0.0,       0.0,       0.0,       0.0, ...),
            (      0.0,       0.0,       0.0,       0.0,       0.0, ...),
            (      0.0,       0.0,       0.0,       0.0,       0.0, ...),
            (      0.0,       0.0,       0.0,       0.0,       0.0, ...),
            (      0.0,       0.0,       0.0,       0.0,       0.0, ...),
            (      0.0,       0.0,       0.0,       0.0,       0.0, ...),
            (      0.0,       0.0,       0.0,       0.0,       0.0, ...),
            (      0.0,       0.0,       0.0,       0.0,       0.0, ...),
            ...
            (3000000.0, 6000000.0, 7500000.0, 9500000.0, 9500000.0, ...),
            (3000000.0, 6000000.0, 7500000.0, 9500000.0, 9500000.0, ...),
            (3000000.0, 6000000.0, 7500000.0, 9500000.0, 9500000.0, ...),
            (3000000.0

In [114]:
wide

Index,CIGS,CdTe,GaAs,InGaP,Perovskite,Polysilicon,Power Electronics,Soft Costs,Capital,Efficiency,GHG,Hazardous,LCOE,Lifetime,Strategic,Yield
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.575208,2.065006,-0.003592,0.975104,-0.143805,187.990992,0.063460,10010.621752
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,-1.486433,2.065172,-0.003592,0.974886,-0.141027,187.990984,0.063460,10010.620799
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,-1.319666,2.065531,-0.003592,0.974471,-0.135810,187.990989,0.063460,10010.619809
3,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,0.0,-1.505137,2.064491,-0.003592,0.963859,-0.137940,187.990949,0.063460,10073.956309
4,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,1000000.0,-1.416362,2.064657,-0.003592,0.963641,-0.135162,187.990941,0.063460,10073.955357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6556,3000000.0,6000000.0,7500000.0,9500000.0,9500000.0,4500000.0,1000000.0,1000000.0,3.288859,2.336959,-0.003592,0.799984,0.055600,188.011181,0.042934,10282.652652
6557,3000000.0,6000000.0,7500000.0,9500000.0,9500000.0,4500000.0,1000000.0,5000000.0,3.455626,2.337318,-0.003592,0.799569,0.060818,188.011186,0.042934,10282.651661
6558,3000000.0,6000000.0,7500000.0,9500000.0,9500000.0,4500000.0,5000000.0,0.0,3.290173,2.338137,-0.003592,0.791527,0.057757,188.011155,0.042934,10326.240255
6559,3000000.0,6000000.0,7500000.0,9500000.0,9500000.0,4500000.0,5000000.0,1000000.0,3.378947,2.338304,-0.003592,0.791309,0.060534,188.011147,0.042934,10326.239303


**Input to MIP constructor needed**: List of investment categories

**Input to MIP constructor needed**: List of metrics to optimize and/or constrain

**Data check**: Confirm that all elements of both input lists match columns in the `evaluate_corners_wide()` data frame.

In [130]:
categories = ['Soft Costs', 'Perovskite', 'CIGS']

metrics = ['Capital']

Fill in the various index values for the one-by-one data set.

In [131]:
# Number of investment categories
Inv = len(categories)

# Number of metrics
J = len(metrics)

In [132]:
# Named series of the number of elicited funding levels in each investment category of interest
l = pd.Series(data=wide.nunique(axis=0, dropna=True)[categories], index=categories)

# Named series of the number of linear intervals in each investment category of interest
n = pd.Series(data=[x - 1 for x in l], index=categories)

# Named series of the number of new lambda variables in each investment category of interest
k = l.copy()

Pull out the investment values and metric values from the data set.

In [136]:
# Investment values
v = wide.loc[:,categories]

# Elicited metric values
m = wide.loc[:, metrics]

**Question**: Is there a standard way we're setting up the budget constraint(s)? e.g. Will there always be as many budget constraints as there are investment categories or always only one budget constraint?

Define upper bound(s) for budget constraint(s).

In [137]:
B = 3000000.0

Instantiate the MIP optimization problem.

In [138]:
example = Model()
bin_vars = []
lmbd_vars = []

Create binary (integer) variables $y_{in_i}$.

In [140]:
for i in range(Inv):
    for n_i in range(n[i]):
        _name = 'y' + '_' + str(i) + '_' + str(n_i)
        bin_vars += [_name]
        example.add_var(name=_name, var_type=BINARY)

Create continuous $\lambda$ variables with lower bound 0.0 and upper bound 1.0

In [141]:
for i in range(Inv):
    for k_i in range(k[i]):
        _name = 'lmbd' + '_' + str(i) + '_' + str(k_i)
        lmbd_vars += [_name]
        example.add_var(name=_name, lb=0.0, ub=1.0)

**In progress/Not implemented from this point onward**

Create budget constraint as a function of the $\lambda$ variables and the elicited investment levels from the data set.

In [66]:
example += sum(xsum(lmbd[i] * v[i] for i in categories)) <= B

NameError: name 'lmbd' is not defined

Convexity constraints on $\lambda$ variables.

In [19]:
example += sum(lmbd_1) == 1

Constrain binary $y$ variables such that at most one of the $y$ variables can be equal to 1.

In [20]:
example += sum(y_1) == 1

Interval constraints on $y$ variables and $\lambda$ variables.

In [21]:
example += y_1[0] <= lmbd_1[0] + lmbd_1[1]

In [22]:
example += y_1[1] <= lmbd_1[1] + lmbd_1[2]

Create objective function: Capital (metric) as a function of $\lambda$s and $y$s.

In [23]:
example.objective = minimize(-1.0 * xsum(lmbd_1[i] * m_1[i] for i in range(k_1)))

Optimize.

In [24]:
example.optimize()

<OptimizationStatus.OPTIMAL: 0>

Print optimal objective function value, with a negative applied to reverse the -1.0 in the objective function definition (minimizing a negative -> maximize a positive).

In [25]:
-1.0 * example.objective_value

-1.3555357403466024

Get the optimal $\lambda$ and $y$ values.

In [26]:
for v in example.vars:
    print('{} : {}'.format(v.name, v.x))

y_1 : 0.0
y_1 : 1.0
lmbd_1 : 0.0
lmbd_1 : 0.5
lmbd_1 : 0.5


\$3,000,000 is halfway between the second and third investment levels, so this solution is the correct optimum.